# Download Packages

In [ ]:
%%capture
!pip install -U git+https://github.com/UN-GCPDS/python-gcpds.databases #Package for database reading.
!pip install mne #The MNE Package is installed
FILEID = "1-bPsREsUCOiJHzIqi8DQrfSjTAf5VAW_"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O MI_EEG_ClassMeth.zip && rm -rf /tmp/cookies.txt
!unzip MI_EEG_ClassMeth.zip #Package with useful functions for motor imagery classification based in EEG.
!dir

# Custom Models

In [ ]:
import tensorflow as tf

def generate_upper_triangular_mask(X):
  """
  Generate a boolean mask to extract the upper triangular part of a matrix.
  """
  ones = tf.ones_like(X)
  mask_a = tf.linalg.band_part(ones, 0, -1) #Upper triangular matrix of 0s and 1s
  mask_b = tf.linalg.band_part(ones, 0, 0)  #Diagonal matrix of 0s and 1s
  mask = tf.cast(mask_a - mask_b, dtype = tf.bool) #Make a bool mask
  return mask


from tensorflow.keras.layers import Layer
import tensorflow as tf
import tensorflow_probability as tfp

class GFC(Layer):
  def __init__(self, **kwargs):
    super().__init__(**kwargs)

  def build(self, batch_input_shape):
    self.gammad = self.add_weight(name = 'gammad',
                            shape = (),
                            initializer = 'zeros',
                            trainable = True)
    super().build(batch_input_shape)

  def call(self, X):
    X = tf.transpose(X, perm  = (0, 3, 1, 2)) #(N, F, C, T)
    R = tf.reduce_sum(tf.math.multiply(X, X), axis = -1, keepdims = True) #(N, F, C, 1)
    D  = R - 2*tf.matmul(X, X, transpose_b = (0, 1, 3, 2)) + tf.transpose(R, perm = (0, 1, 3, 2)) #(N, F, C, C)

    mask = generate_upper_triangular_mask(D[0,0,...]) #(C, C)
    triu = tf.expand_dims(tf.boolean_mask(D, mask, axis = 2), axis = -1) #(N, F, C*(C-1)/2, 1)
    sigma = tfp.stats.percentile(tf.math.sqrt(triu), 50, axis = 2, keepdims = True) #(N, F, 1, 1)

    A = tf.math.exp(-1/(2*tf.pow(10., self.gammad)*tf.math.square(sigma))*D) #(N, F, C, C)
    A.set_shape(D.shape)
    return A

  def compute_output_shape(self, batch_input_shape):
    N, C, T, F = batch_input_shape.as_list()
    return tf.TensorShape([N, F, C, C])

  def get_config(self):
    base_config = super().get_config()
    return {**base_config}

from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, DepthwiseConv2D, Activation, AveragePooling2D, Dropout, SpatialDropout2D, SeparableConv2D, Flatten, Dense
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.models import Model

def FC_EEGNet(number_of_classes: int,
          Chans: int,
          Samples: int,
          dropout_rate: float,
          kernLength: int,
          F1: int,
          D: int,
          F2: int,
          norm_rate: int,
          dropout_type: str) -> Model:
    """ Keras Implementation of EEGNet
    http://iopscience.iop.org/article/10.1088/1741-2552/aace8c/meta
    Note that this implements the newest version of EEGNet and NOT the earlier
    version (version v1 and v2 on arxiv). We strongly recommend using this
    architecture as it performs much better and has nicer properties than
    our earlier version. For example:

        1. Depthwise Convolutions to learn spatial filters within a
        temporal convolution. The use of the depth_multiplier option maps
        exactly to the number of spatial filters learned within a temporal
        filter. This matches the setup of algorithms like FBCSP which learn
        spatial filters within each filter in a filter-bank. This also limits
        the number of free parameters to fit when compared to a fully-connected
        convolution.

        2. Separable Convolutions to learn how to optimally combine spatial
        filters across temporal bands. Separable Convolutions are Depthwise
        Convolutions followed by (1x1) Pointwise Convolutions.


    While the original paper used Dropout, we found that SpatialDropout2D
    sometimes produced slightly better results for classification of ERP
    signals. However, SpatialDropout2D significantly reduced performance
    on the Oscillatory dataset (SMR, BCI-IV Dataset 2A). We recommend using
    the default Dropout in most cases.

    Assumes the input signal is sampled at 128Hz. If you want to use this model
    for any other sampling rate you will need to modify the lengths of temporal
    kernels and average pooling size in blocks 1 and 2 as needed (double the
    kernel lengths for double the sampling rate, etc). Note that we haven't
    tested the model performance with this rule so this may not work well.

    The model with default parameters gives the EEGNet-8,2 model as discussed
    in the paper. This model should do pretty well in general, although it is
	advised to do some model searching to get optimal performance on your
	particular dataset.
    We set F2 = F1 * D (number of input filters = number of output filters) for
    the SeparableConv2D layer. We haven't extensively tested other values of this
    parameter (say, F2 < F1 * D for compressed learning, and F2 > F1 * D for
    overcomplete). We believe the main parameters to focus on are F1 and D.
    Inputs:

      nb_classes      : int, number of classes to classify
      Chans, Samples  : number of channels and time points in the EEG data
      dropout_rate     : dropout fraction
      kernLength      : length of temporal convolution in first layer. We found
                        that setting this to be half the sampling rate worked
                        well in practice. For the SMR dataset in particular
                        since the data was high-passed at 4Hz we used a kernel
                        length of 32.
      F1, F2          : number of temporal filters (F1) and number of pointwise
                        filters (F2) to learn. Default: F1 = 8, F2 = F1 * D.
      D               : number of spatial filters to learn within each temporal
                        convolution. Default: D = 2
      dropout_type     : Either SpatialDropout2D or Dropout, passed as a string.
    """

    if dropout_type == 'SpatialDropout2D':
        dropout_type = SpatialDropout2D
    elif dropout_type == 'Dropout':
        dropout_type = Dropout
    else:
        raise ValueError('dropout_type must be one of SpatialDropout2D '
                         'or Dropout, passed as a string.')

    input_   = Input(shape = (Chans, Samples, 1))

    ##################################################################
    #Temporal Convolution
    block1       = Conv2D(F1, (1, kernLength), padding = 'same',
                          name = 'Temporal_Conv2D',
                          use_bias = False)(input_)
    block1       = BatchNormalization()(block1)
    adj_mat      = GFC(name = 'gfc')(block1)

    ##################################################################
    #Spatial Convolution
    block1       = DepthwiseConv2D((Chans, 1),
                                   name = 'Spatial_Depth_wise_Conv2D',
                                   depth_multiplier = D,
                                   use_bias = False,
                                   depthwise_constraint = max_norm(1.))(block1)
    block1       = BatchNormalization()(block1)
    block1       = Activation('elu')(block1)
    block1       = AveragePooling2D((1, 4))(block1)
    block1       = dropout_type(dropout_rate)(block1)

    ##################################################################
    #Separable Convolution
    block2       = SeparableConv2D(F2, (1, 16), padding = 'same',
                                   name = 'Separable_Conv2D',
                                   use_bias = False)(block1)
    block2       = BatchNormalization()(block2)
    block2       = Activation('elu')(block2)
    block2       = AveragePooling2D((1, 8))(block2)
    block2       = dropout_type(dropout_rate)(block2)

    ##################################################################
    # Classification block
    flatten      = Flatten(name = 'flatten')(block2)
    dense        = Dense(number_of_classes, name = 'output',
                         kernel_constraint = max_norm(norm_rate))(flatten)
    softmax      = Activation('softmax', name = 'out_activation')(dense)

    return Model(inputs=input_, outputs = [softmax, adj_mat])

# Functions

In [ ]:
from gcpds.databases.BCI_Competition_IV import Dataset_2a
from typing import Sequence, Tuple
from MI_EEG_ClassMeth.FeatExtraction import TimeFrequencyRpr
import numpy as np
from scipy.signal import resample

def load_BCICIV2a(db: Dataset_2a,
               sbj: int,
               mode: str,
               fs: float,
               f_bank: np.ndarray,
               vwt: np.ndarray,
               new_fs: float) -> np.ndarray:

  tf_repr = TimeFrequencyRpr(sfreq = fs, f_bank = f_bank, vwt = vwt)

  db.load_subject(sbj, mode = mode)
  X, y = db.get_data() #Load all classes, all channels {EEG, EOG}, reject bad trials
  X = X[:,:-3,:] # pick EEG channels
  X = X*1e6 #uV
  X = np.squeeze(tf_repr.transform(X))
  #Resampling
  if new_fs == fs:
    print('No resampling, since new sampling rate same.')
  else:
    print("Resampling from {:f} to {:f} Hz.".format(fs, new_fs))
    X = resample(X, int((X.shape[-1]/fs)*new_fs), axis = -1)

  return X, y


from gcpds.databases import GIGA_MI_ME

def load_GIGA_MI_ME(db: GIGA_MI_ME,
              sbj: int,
              eeg_ch_names: Sequence[str],
              fs: float,
              f_bank: np.ndarray,
              vwt: np.ndarray,
              new_fs: float) -> Tuple[np.ndarray, np.ndarray]:

  index_eeg_chs = db.format_channels_selectors(channels = eeg_ch_names) - 1

  tf_repr = TimeFrequencyRpr(sfreq = fs, f_bank = f_bank, vwt = vwt)

  db.load_subject(sbj)
  X, y = db.get_data(classes = ['left hand mi', 'right hand mi']) #Load MI classes, all channels {EEG}, reject bad trials, uV
  X = X[:, index_eeg_chs, :] #spatial rearrangement
  X = np.squeeze(tf_repr.transform(X))
  #Resampling
  if new_fs == fs:
    print('No resampling, since new sampling rate same.')
  else:
    print("Resampling from {:f} to {:f} Hz.".format(fs, new_fs))
    X = resample(X, int((X.shape[-1]/fs)*new_fs), axis = -1)

  return X, y


def load_DB(db_name, **load_args):
  if db_name == 'BCICIV2a':
    X_train, y_train = load_BCICIV2a(**load_args, mode = 'training')
    X_test, y_test = load_BCICIV2a(**load_args, mode = 'evaluation')

    X_train = np.concatenate([X_train, X_test], axis = 0)
    y_train = np.concatenate([y_train, y_test], axis = 0)

  elif db_name == 'GIGA_MI_ME':
    X_train, y_train = load_GIGA_MI_ME(**load_args)

  else:
    raise ValueError('No valid database name')

  return X_train, y_train

from scipy.stats import ks_2samp
from scipy.spatial.distance import squareform

def KS_matrix(A: np.ndarray, y: np.ndarray) -> np.ndarray:
  """
  Input
  -----
  A Tensor (batch_dim, n_frquency_bands, n_chans, n_chans)
  Output
  ------
  Q Tensor(n_frquency_bands, n_chans, n_chans)
  """
  _, F, C, _ = A.shape
  utri_ind =  np.triu_indices(C, 1)
  A = A[:, :, utri_ind[0], utri_ind[1]] #Flatten
  D = A.shape[-1]
  A_l = A[y == 0]
  A_r = A[y == 1]
  Q = np.zeros((F, C, C))

  for f in range(F):
    Q_ = np.zeros(D)
    for d in range(D):
      Q_[d], _ = ks_2samp(A_l[:, f, d], A_r[:, f, d], alternative = 'two-side', mode = 'auto')
    Q[f] = squareform(Q_)

  return Q

def all_argmmax(x):
    if x.shape[0] == 0:
        return []
    all_ = [0]
    max_ = x[0]
    for i in range(1, x.shape[0]):
        if x[i] > max_:
            all_ = [i]
            max_ = x[i]
        elif x[i] == max_:
            all_.append(i)
    return all_

def relevant_frequencies(A, trh):
  rel_freq = []

  utri_ind =  np.triu_indices(A.shape[-1], 1)
  A = A[:, utri_ind[0], utri_ind[1]] #Flatten
  A[A < trh] = 0

  for d in range(A.shape[-1]):
    if A[:,d].sum() != 0:
      rel_freq += all_argmmax(A[:,d])

  return rel_freq

def renyis_entropy_2(A):
  """
  Renyis entropy with alpha = 2
  Input
  -----
  A Tensor (batch_dim, n_freq_bands, n_chans, n_chans)
  Output
  ------
  H Tensor (batch_dim, n_freq_bands)
  """
  A = A/A.shape[-1]
  H = -1*tf.math.log(tf.linalg.trace(tf.linalg.matmul(A ,A)))
  H = H.numpy()
  H = H.flatten()
  return H

# Import Drive

In [ ]:
from google.colab.drive import mount
mount("/content/drive", force_remount=True)

Mounted at /content/drive


#Config

In [ ]:
db_name = 'BCICIV2a'
nb_classes = 4
model_params = dict(number_of_classes = nb_classes,
              dropout_rate = 0.5,
              kernLength = 32,
              F1 = 8,
              D = 2,
              F2 = 16,
              norm_rate = 0.25,
              dropout_type = 'Dropout')
parent_dir = '/content/drive/Shareddrives/GCPDS - Mateo/Mateo/MI_DL_models/' + db_name + '/'

In [ ]:
import numpy as np

if db_name == 'BCICIV2a':
  db = Dataset_2a('/content/drive/Shareddrives/GCPDS-Databases/BCI_Competition_IV/dataset_2a')
  fs = db.metadata['sampling_rate']
  load_args = dict(db = db,
                 fs = fs,
                 f_bank = np.asarray([[4., 40.]]),
                 vwt = np.asarray([[2.5, 6]]),
                 new_fs = 128.)
  subjects = np.arange(db.metadata['subjects']) + 1

elif db_name == 'GIGA_MI_ME':
  db = GIGA_MI_ME('/content/drive/Shareddrives/GCPDS-Databases/GIGA-MI_ME/')
  fs = db.metadata['sampling_rate']
  eeg_ch_names = ['Fp1','Fpz','Fp2',
                'AF7','AF3','AFz','AF4','AF8',
                'F7','F5','F3','F1','Fz','F2','F4','F6','F8',
                'FT7','FC5','FC3','FC1','FCz','FC2','FC4','FC6','FT8',
                'T7','C5','C3','C1','Cz','C2','C4','C6','T8',
                'TP7','CP5','CP3','CP1','CPz','CP2','CP4','CP6','TP8',
                'P9','P7','P5','P3','P1','Pz','P2','P4','P6','P8','P10',
                'PO7','PO3','POz','PO4','PO8',
                'O1','Oz','O2',
                'Iz']

  load_args = dict(db = db,
                  eeg_ch_names = eeg_ch_names,
                  fs = fs,
                  f_bank = np.asarray([[4., 40.]]),
                  vwt = np.asarray([[2.5, 5]]),
                  new_fs = 128.)
  subjects = np.arange(db.metadata['subjects']) + 1
  subjects = np.delete(subjects, [28,33])

else:
  raise ValueError('No valid database name')

# Main

In [ ]:
from pickle import load as pload
from pickle import dump
import pandas as pd

### RCKA_EEGNet

In [ ]:
%%time
metrics = ['acc', 'std_acc', 'kappa', 'kappa_std', 'auc', 'std_auc']
C = len(db.metadata['channels'])
if db_name == 'BCICIV2a':
  C -= 3 #Remove EOG channels
utri_ind =  np.triu_indices(C, 1)


results = np.zeros((subjects.shape[0], len(metrics)))
lambda_ = np.zeros((subjects.shape[0]))
gammad = np.zeros((subjects.shape[0]))
H = np.zeros((subjects.shape[0]))
rel_freq = []
A_sbjs = np.zeros((subjects.shape[0], int(C*(C-1)*0.5)))

for idx, sbj in enumerate(subjects):
  with open(parent_dir + 'RCKA_EEGNet/raw_results/sbj' + str(sbj) + '.txt', 'rb') as f:
    sbj_results = pload(f)

  #Classification performance metrics, hyperparameters
  best_index = sbj_results['best_index']
  results[idx] = sbj_results['mean_acc'][best_index], sbj_results['std_acc'][best_index], sbj_results['mean_kappa'][best_index], sbj_results['std_kappa'][best_index], sbj_results['mean_auc'][best_index], sbj_results['std_auc'][best_index]
  lambda_[idx] = sbj_results['params'][best_index]

  #Model paramas
  load_args['sbj'] = sbj
  X_train, y_train = load_DB(db_name, **load_args)
  T = X_train.shape[-1]
  X_train = X_train[..., np.newaxis]
  model = FC_EEGNet(**model_params, Chans = C, Samples = T)
  model.load_weights(parent_dir + 'RCKA_EEGNet/models/' + str(sbj_results['params'][best_index]) + '_sbj' + str(sbj) + '.h5')
  gfc_layer = model.get_layer('gfc')
  gammad[idx] = gfc_layer.get_weights()[0]
  A = model.predict(X_train)[-1] #(N, F, C, C)

  #Results for model interpretation
  A_k = KS_matrix(A, y_train) #(F, C, C)
  rel_freq.append(relevant_frequencies(A_k, 0.5))
  H[idx] = renyis_entropy_2(np.max(A_k, axis = 0))
  A_sbjs[idx] = np.max(A_k[:, utri_ind[0], utri_ind[1]], axis = 0)

#Save results
np.save(parent_dir + 'RCKA_EEGNet/reg_eegnet', results)
np.save(parent_dir + 'RCKA_EEGNet/lambda_', lambda_)
np.save(parent_dir + 'RCKA_EEGNet/gammad', gammad)
np.save(parent_dir + 'RCKA_EEGNet/H', H)
with open(parent_dir + 'RCKA_EEGNet/rel_freq.txt', 'wb') as f:
    dump(rel_freq, f)
np.save(parent_dir + 'RCKA_EEGNet/A_sbjs', A_sbjs)

Resampling from 250.000000 to 128.000000 Hz.
Resampling from 250.000000 to 128.000000 Hz.
18/18 [==============================] - 8s 9ms/step
Resampling from 250.000000 to 128.000000 Hz.
Resampling from 250.000000 to 128.000000 Hz.
18/18 [==============================] - 0s 8ms/step
Resampling from 250.000000 to 128.000000 Hz.
Resampling from 250.000000 to 128.000000 Hz.
17/17 [==============================] - 0s 9ms/step
Resampling from 250.000000 to 128.000000 Hz.
Resampling from 250.000000 to 128.000000 Hz.
16/16 [==============================] - 0s 5ms/step
Resampling from 250.000000 to 128.000000 Hz.
Resampling from 250.000000 to 128.000000 Hz.
17/17 [==============================] - 0s 11ms/step
Resampling from 250.000000 to 128.000000 Hz.
Resampling from 250.000000 to 128.000000 Hz.
14/14 [==============================] - 0s 10ms/step
Resampling from 250.000000 to 128.000000 Hz.
Resampling from 250.000000 to 128.000000 Hz.
18/18 [==============================] - 0s 11ms/s

### EGGNet

In [ ]:
%%time
results = np.zeros((subjects.shape[0], len(metrics)))

for idx, sbj in enumerate(subjects):
  with open(parent_dir + 'EEGNet/raw_results/sbj' + str(sbj) + '.txt', 'rb') as f:
    sbj_results = pload(f)

  #Classification performance metrics, hyperparameters
  results[idx] = sbj_results['mean_acc'], sbj_results['std_acc'], sbj_results['mean_kappa'], sbj_results['std_kappa'], sbj_results['mean_auc'], sbj_results['std_auc']

np.save(parent_dir + 'EEGNet/raw_eegnet', results)

CPU times: user 28.1 ms, sys: 3.7 ms, total: 31.8 ms
Wall time: 7.29 s
